# LangChain と Gemini API で PDF ドキュメントの要約を行う例

LangChain のパッケージ、および、PDF の扱いに必要なパッケージをインストールします。

In [ ]:
!pip install --user \
  langchain==0.1.0 transformers==4.36.0 \
  pypdf==3.17.0 cryptography==42.0.4 \
  langchain-google-vertexai==0.0.6 \
  google-cloud-aiplatform==1.42.1

**注意：次のセルを実行する前にカーネルをリスタートしてください。**

In [ ]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

日本人間ドック・予防医療学会 が一般公開している「[検査表の見方](https://www.ningen-dock.jp/public_method/)」をダウンロードします。

In [ ]:
base_url = 'https://raw.githubusercontent.com/gongqi-zhen/HealthBuddy-QA/main'
!wget -q $base_url/PDF/Japan_Society_of_Ningen_Dock_and_Preventive_Medical_Care/shoken-mikata.pdf

ダウンロードした PDF を読み込んで、全体を1つのテキストにまとめます。

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pages = PyPDFLoader('shoken-mikata.pdf').load()
document = ''
for page in pages:
    document += page.page_content

Gemini API を利用するクライアントオブジェクトを取得します。

In [ ]:
from langchain_google_vertexai import VertexAI
llm = VertexAI(model_name='gemini-1.5-flash-001', location='asia-northeast1',
               temperature=0.1, max_output_tokens=1024)

RecursiveCharacterTextSplitter, load_qa_chain, AnalyzeDocumentChain モジュールの使用例です。

`input_document` に与えたテキストに基づいて、`question` の質問に回答します。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='map_reduce')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

output = qa_document_chain.invoke(
    {'input_document':'今は６月で雨が多い時期です。', 'question':'最近の天候は？'})
print(output)

`input_document` に与えたテキストの内容をまとめるように指示して結果を得る関数を用意します。

In [ ]:
def get_description(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=6000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='map_reduce')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '何についての文書ですか？日本語で200字程度にまとめて教えてください。'
    description = qa_document_chain.invoke(
        {'input_document': document, 'question': prompt})
    return description['output_text'].replace('FINAL ANSWER: ', '')

先に用意した PDF ファイルを適用します。

In [ ]:
print(get_description(document))

任意の質問に答える関数を用意します。

In [ ]:
def get_answer(document, question):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='refine')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '{} 日本語で200字程度にまとめて教えてください。マークダウンを使用せずにプレーンテキストで出力。'.format(question)
    answer = qa_document_chain.invoke(
        {'input_document': document, 'question': prompt})
    return answer['output_text']

先に用意した PDF ファイルに基づいて、質問の回答を取得します。

In [ ]:
question = '人間ドックの検査項目とその結果の見方についてのポイントを箇条書きにまとめてください。'
print(get_answer(document, question))

In [ ]:
question = '生活習慣病について教えてください。BMI値が30でした。元気が出る言葉を添えて励ましつつ、改善活動案を３つ示してください。'
print(get_answer(document, question))

In [ ]:
question = '糖尿病について教えてください。空腹時血糖が126でHbA1cが6.6%でした。元気が出る言葉を添えて励ましつつ、改善活動案を３つ示してください。'
print(get_answer(document, question))

In [ ]:
question = '高脂血症リスクについて教えてください。LDL140でHDL34で中性脂肪が301でした。元気が出る言葉を添えて励ましつつ、改善活動案を３つ示してください。'
print(get_answer(document, question))

In [ ]:
question = '高血圧リスクについて教えてください。収縮期血圧が129で拡張期血圧が84でした。元気が出る言葉を添えて励ましつつ、改善活動案を３つ示してください。'
print(get_answer(document, question))

In [ ]:
question = '肝機能リスクについて教えてください。GOTが51でγGTPが102でした。元気が出る言葉を添えて励ましつつ、改善活動案を３つ示してください。'
print(get_answer(document, question))

In [ ]:
question = '''\
健康診断の結果

BMI: 25
腹囲: 85cm
空腹時血糖: 100
HbA1c: 5.6%
LDL: 120
HDL: 39
中性脂肪: 150
収縮期血圧: 130
拡張期血圧: 85
GOT: 31
GPT: 31
γGTP: 51

でした。

この結果から検査項目とその結果の見方についてのポイントを教えてください。

保健指導を受けて、生活習慣を改善したいです。

また、最近、疲れ目と、腰痛があります。

胸部X線所見に異常ありと書かれてあります。

元気が出る言葉を添えて励ましつつ、改善活動案を３つ示してください。
'''

print(get_answer(document, question))